In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a
    from mu.app_daily
    where 
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 01, 31)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 03, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 03, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                        reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity={raw_granularity}/" \
                   "month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy('date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(start_day=month_day_list_tuple[1][0], end_day=month_day_list_tuple[1][len(month_day_list_tuple[1])-1]))
        # print month_day_list_tuple[1]
        # print dump_count_list
        # print plproxy_count_list
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}".format(dump_count['date'])
        break

granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

from pyspark.sql.functions import count
df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/")
print df.groupBy('date').agg(count('app_id').alias('count')).collect()

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta



test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a
    from mu.app_daily
    where 
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 01, 31)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 03, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 03, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                        reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    for month_day_list_tuple in date_list:
        plproxy_count_list = query(aa_dsn, sql.format(start_day=month_day_list_tuple[1][0], end_day=month_day_list_tuple[1][len(month_day_list_tuple)-1]))
        print plproxy_count_list
        break


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a
    from mu.app_daily
    where 
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 01, 31)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 03, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                        reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/" \
                    "granularity={raw_granularity}/month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy('date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(
            start_day=month_day_list_tuple[1][0], end_day=month_day_list_tuple[1][len(month_day_list_tuple[1])-1]))
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            test_result.append((_granularity, dump_count['date'], plproxy_count[1], dump_count['count']))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'plproxy_count', 'dump_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0605/daily/",
            mode="append",
            partitionBy=["type"])
    retry(write_test_result, (df_write_result,), {}, interval=10)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'


In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a
    from mu.app_{}
    where 
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                        reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/" \
                    "granularity={raw_granularity}/month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy('date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(_granularity, start_day=month_day_list_tuple[1][0],
                                                      end_day=month_day_list_tuple[1][len(month_day_list_tuple[1])-1]))
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            test_result.append((_granularity, dump_count['date'], plproxy_count[1], dump_count['count']))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'plproxy_count', 'dump_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0605/weekly/",
            mode="overwrite",
            partitionBy=["type"])
    retry(write_test_result, (df_write_result,), {}, interval=10)


granularity_list = ["weekly"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'


In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a
    from mu.app_{}
    where 
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                        reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/" \
                    "granularity={raw_granularity}/month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy('date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(_granularity, start_day=month_day_list_tuple[1][0],
                                                      end_day=month_day_list_tuple[1][len(month_day_list_tuple[1])-1]))
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            test_result.append((_granularity, dump_count['date'], plproxy_count[1], dump_count['count']))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'plproxy_count', 'dump_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0605/monthly/",
            mode="overwrite",
            partitionBy=["type"])
    retry(write_test_result, (df_write_result,), {}, interval=10)


granularity_list = ["monthly"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

df = spark.read.format("delta").load('s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0605/monthly/').orderBy('date').collect()
for row in df:
    print row['date'],'\t',int(row['plproxy_count']),'\t',row['dump_count']

In [0]:

# Copyright (c) 2020 App Annie Inc. All rights reserved.

from aadatapipelinecore.core.urn import Urn
import psycopg2
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


def get_path_date_list(granularity):
    if granularity == 'daily':
        date_list = [('2015-12', '2015-12-27'), ('2016-06', '2016-06-27'),
                     ('2016-12', '2016-12-27'), ('2017-06', '2017-06-27'),
                     ('2017-12', '2017-12-27')]
    if granularity == 'weekly':
        date_list = [('2013-12', '2013-12-28'), ('2014-12', '2014-12-27'),
                     ('2015-12', '2015-12-26'), ('2016-12', '2016-12-24'),
                     ('2017-12', '2017-12-23')]
    if granularity == 'monthly':
        date_list = [('2013-12', '2013-12-31'), ('2014-12', '2014-12-31'),
                     ('2015-12', '2015-12-31'), ('2016-12', '2016-12-31'),
                     ('2017-12', '2017-12-31')]
    return date_list


PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select device_id,store_id,date,app_id,kpi,estimate from plproxy.execute_select_nestloop($proxy$
    select device_id,store_id,date,app_id,kpi,estimate
    from mu.app_{}
    where
        date = '{}'
$proxy$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) limit 50;"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_plproxy_dump_accuracy(date_list, graularity):
    """
        date_list:
                [(month1,day1), (month1,day2), (month2,day1), (month2,day2)]
        sample:
            [('2015-12', '2015-12-27'), ('2015-12', '2015-12-28'),
            ('2016-12', '2016-12-27'), ('2016-12', '2016-12-28')]
    """
    for date in date_list:
        result = query(aa_dsn, sql.format(graularity, date[1]))
        plproxy_df = spark.createDataFrame(result,
                                           schema=['device_id', 'store_id', 'date', 'app_id', 'kpi', 'estimate'])

        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/" \
                    "granularity={}/month={}/"
        dump_df = spark.read.parquet(dump_path.format(graularity, date[0]))

        join_df_count = dump_df.filter("date='{}'".format(date[1])).join(
            plproxy_df, on=[plproxy_df.device_id == dump_df.device_id, plproxy_df.store_id == dump_df.store_id,
                            plproxy_df.app_id == dump_df.app_id, plproxy_df.kpi == dump_df.kpi,
                            plproxy_df.estimate == dump_df.estimate], how='inner').count()
        if join_df_count != 50:
            print "Accuracy Test Fail!!!! graularity: {}, date: {}".format(graularity, date[1])
        else:
            print "Accuracy Test Pass! graularity: {},date : {}".format(graularity, date[1])


graularity_list = ["daily", "weekly", "monthly"]
for graularity in graularity_list:
    check_plproxy_dump_accuracy(get_path_date_list(graularity), graularity)


In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2019-10-01/')
df.filter("app_id=20600001429645 and country_code='CN' and device_code='android-phone'").select('est_average_bytes_per_user').show()

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
kpi_mapping_dict = {1: 'AU', 2: 'AFU', 3: 'ADU', 4: 'IP', 5: 'AAD', 6: 'PAD', 7: 'MBPU',
                    8: 'ATU', 9: 'UP', 10: 'OR', 17: 'MBPS', 23: 'IS', 24: 'SOU', 25: 'SOI'}
granularity_dict = {'daily': 'DAY', 'monthly': 'MONTH', 'weekly': 'WEEK'}
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select kpi, sum(kpi_count) from plproxy.execute_select_nestloop($proxy$ 
    select kpi, count(app_id) as kpi_count
    from mu.app_{}
    where 
        date = '{}'
    group by kpi
$proxy$) t (kpi SMALLINT, kpi_count BIGINT) group by kpi order by kpi asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 3, 31)
    start = datetime.date(2020, 3, 1)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    date_list = [(x[0][:7], x[0]) for x in collect_date]
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_routine_consistency(_granularity, date_list):
    routine_path = 's3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type={}/date={}/'
    for date in date_list:
        plproxy_kpi_count_list = query(aa_dsn, sql.format(_granularity, date[1]))
        routine_path_parse = routine_path.format(granularity_dict[_granularity], date[1])
        routine_df = spark.read.parquet(routine_path_parse)
        for plproxy_kpi_count in plproxy_kpi_count_list:
            routine_kpi_count = routine_df.filter(
                "{} is not null".format(kpi_mapping_dict[plproxy_kpi_count[0]])).count()
            if plproxy_kpi_count[1] != routine_kpi_count:
                print "Consistency Test Fail!!!! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count[1], routine_kpi_count, kpi_mapping_dict[plproxy_kpi_count[0]]
                )
            else:
                print "Consistency Test Pass! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count[1], routine_kpi_count, kpi_mapping_dict[plproxy_kpi_count[0]]
                )



granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_routine_consistency(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from conf import settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
kpi_mapping_dict = {1: 'est_average_active_users', 2: 'est_average_session_per_user', 3: 'est_average_session_duration',
                    4: 'est_install_penetration', 5: 'est_average_active_days', 6: 'est_percentage_active_days',
                    7: 'est_average_bytes_per_user', 8: 'est_average_time_per_user', 9: 'est_usage_penetration',
                    10: 'est_open_rate', 17: 'est_average_bytes_per_session', 23: 'est_installs', 24: 'est_share_of_users',
                    25: 'est_share_of_installs'}
granularity_dict = {'daily': 'DAY', 'monthly': 'MONTH', 'weekly': 'WEEK'}
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select kpi, sum(kpi_count) from plproxy.execute_select_nestloop($proxy$ 
    select kpi, count(app_id) as kpi_count
    from mu.app_{}
    where 
        date = '{}'
    group by kpi
$proxy$) t (kpi SMALLINT, kpi_count BIGINT) group by kpi order by kpi asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 5, 23)
    start = datetime.date(2020, 5, 17)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    date_list = [(x[0][:7], x[0]) for x in collect_date]
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_routine_consistency(_granularity, date_list):
    plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/'
    routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/'
    for date in date_list:
        plproxy_path_parse = plproxy_path.format(_granularity, date[1])
        routine_path_parse = routine_path.format(_granularity, date[1])
        plproxy_df = spark.read.format("delta").load(plproxy_path).where("granularity='daily' and date='{}'".format(date[1]))
        routine_df = spark.read.format("delta").load(routine_path).where("granularity='daily' and date='{}'".format(date[1]))
        for kpi in kpi_mapping_dict:
            if kpi==1:
                plproxy_kpi_count = plproxy_df.filter(
                    "{}!=0".format(kpi_mapping_dict[kpi])).count()
                routine_kpi_count = routine_df.filter(
                    "{}!=0".format(kpi_mapping_dict[kpi])).count()
            else:
                plproxy_kpi_count = plproxy_df.filter(
                    "est_average_active_users!=0 and {}!=0".format(kpi_mapping_dict[kpi])).count()
                routine_kpi_count = routine_df.filter(
                    "est_average_active_users!=0 and {}!=0".format(kpi_mapping_dict[kpi])).count()
            if plproxy_kpi_count != routine_kpi_count:
                print "Consistency Test Fail!!!! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count, routine_kpi_count, kpi_mapping_dict[kpi]
                )
            else:
                print "Consistency Test Pass! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count, routine_kpi_count, kpi_mapping_dict[kpi]
                )


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_routine_consistency(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from conf import settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
kpi_mapping_dict = {1: 'est_average_active_users', 2: 'est_average_session_per_user', 3: 'est_average_session_duration',
                    4: 'est_install_penetration', 5: 'est_average_active_days', 6: 'est_percentage_active_days',
                    7: 'est_average_bytes_per_user', 8: 'est_average_time_per_user', 9: 'est_usage_penetration',
                    10: 'est_open_rate', 17: 'est_average_bytes_per_session', 23: 'est_installs', 24: 'est_share_of_users',
                    25: 'est_share_of_installs'}
granularity_dict = {'daily': 'DAY', 'monthly': 'MONTH', 'weekly': 'WEEK'}
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select kpi, sum(kpi_count) from plproxy.execute_select_nestloop($proxy$ 
    select kpi, count(app_id) as kpi_count
    from mu.app_{}
    where 
        date = '{}'
    group by kpi
$proxy$) t (kpi SMALLINT, kpi_count BIGINT) group by kpi order by kpi asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 6, 20)
    start = datetime.date(2020, 6, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    date_list = [(x[0][:7], x[0]) for x in collect_date]
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_routine_consistency(_granularity, date_list):
    plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/'
    routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/'
    for date in date_list:
        plproxy_path_parse = plproxy_path.format(_granularity, date[1])
        routine_path_parse = routine_path.format(_granularity, date[1])
        plproxy_df = spark.read.format("delta").load(plproxy_path).where("granularity='daily' and date='{}'".format(date[1]))
        routine_df = spark.read.format("delta").load(routine_path).where("granularity='daily' and date='{}'".format(date[1]))
        for kpi in kpi_mapping_dict:
            if kpi==1:
                plproxy_kpi_count = plproxy_df.filter(
                    "{}!=0".format(kpi_mapping_dict[kpi])).count()
                routine_kpi_count = routine_df.filter(
                    "{}!=0".format(kpi_mapping_dict[kpi])).count()
            else:
                plproxy_kpi_count = plproxy_df.filter(
                    "est_average_active_users!=0 and {}!=0".format(kpi_mapping_dict[kpi])).count()
                routine_kpi_count = routine_df.filter(
                    "est_average_active_users!=0 and {}!=0".format(kpi_mapping_dict[kpi])).count()
            if plproxy_kpi_count != routine_kpi_count:
                print "Consistency Test Fail!!!! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count, routine_kpi_count, kpi_mapping_dict[kpi]
                )
            else:
                print "Consistency Test Pass! date: {}, plproxy_count: {}, routine_count: {}, kpi: {}".format(
                    date[1], plproxy_kpi_count, routine_kpi_count, kpi_mapping_dict[kpi]
                )


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_routine_consistency(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date=2020-05-17/'
routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2020-05-17/'
plproxy_df = spark.read.format("delta").load(plproxy_path)
routine_df = spark.read.format("delta").load(routine_path)
print plproxy_df.filter("est_average_active_users!=0").count()
print routine_df.filter("est_average_active_users!=0").count()
# print plproxy_df.filter("est_average_active_users!=0").select('app_id', 'country_code', 'device_code', 'est_average_active_users').subtract(routine_df.filter("est_average_active_users!=0").select('app_id', 'country_code', 'device_code', 'est_average_active_users')).show()
plproxy_df.filter("est_average_active_users!=0").select('app_id', 'country_code', 'device_code').subtract(routine_df.filter("est_average_active_users!=0").select('app_id', 'country_code', 'device_code')).show()
routine_df.filter("est_average_active_users!=0").select('app_id', 'country_code', 'device_code').subtract(plproxy_df.select('app_id', 'country_code', 'device_code')).show()

In [0]:

plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date=2020-05-17/'
routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2020-05-17/'
plproxy_df = spark.read.format("delta").load(plproxy_path)
routine_df = spark.read.format("delta").load(routine_path)
plproxy_df.createOrReplaceTempView("plproxy_df")
routine_df.createOrReplaceTempView("routine_df")
sql = """
        select
            plproxy_df.app_id, plproxy_df.device_code, plproxy_df.country_code,
            plproxy_df.{kpi} as v3_{kpi},
            v5.{kpi} as v5_{kpi}
        from plproxy_df inner join
            (select app_id, device_code, country_code, {kpi}
             from routine_df
             ) AS v5
        on (plproxy_df.app_id=v5.app_id) and (plproxy_df.device_code=v5.device_code) and (plproxy_df.country_code=v5.country_code) and (plproxy_df.{kpi}!=v5.{kpi})
        """.format(kpi='est_average_active_users')
unified_v1 = spark.sql(sql)
unified_v1.show(3)

In [0]:

plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date=2020-06-17/'
routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2020-05-17/'
plproxy_df = spark.read.format("delta").load(plproxy_path)
routine_df = spark.read.format("delta").load(routine_path)
plproxy_df.createOrReplaceTempView("plproxy_df")
routine_df.createOrReplaceTempView("routine_df")
kpi_mapping_dict = {1: 'est_average_active_users', 2: 'est_average_session_per_user', 3: 'est_average_session_duration',
                    4: 'est_install_penetration', 5: 'est_average_active_days', 6: 'est_percentage_active_days',
                    7: 'est_average_bytes_per_user', 8: 'est_average_time_per_user', 9: 'est_usage_penetration',
                    10: 'est_open_rate', 17: 'est_average_bytes_per_session', 23: 'est_installs', 24: 'est_share_of_users',
                    25: 'est_share_of_installs'}
for i,kpi in kpi_mapping_dict.items():
    plproxy_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
    routine_count =  routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
    subtract_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi).subtract(routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi)).count()
    print plproxy_count, routine_count, subtract_count
    if subtract_count!=0:
        print float(subtract_count) / plproxy_count
    sql = """
            select
                plproxy_df.app_id, plproxy_df.device_code, plproxy_df.country_code,
                plproxy_df.{kpi} as v3_{kpi},
                v5.{kpi} as v5_{kpi}
            from plproxy_df inner join
                (select app_id, device_code, country_code, {kpi}
                 from routine_df
                 ) AS v5
            on (plproxy_df.app_id=v5.app_id) and (plproxy_df.device_code=v5.device_code) and (plproxy_df.country_code=v5.country_code) and (plproxy_df.{kpi}!=v5.{kpi})
            """.format(kpi=kpi)
    unified_v1 = spark.sql(sql)
    # unified_v1 = unified_v1= unified_v1.withColumn('difference', unified_v1['v3_{}'.format(kpi)] - unified_v1['v5_{}'.format(kpi)])
    unified_v1.show(3)

In [0]:

routine_df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2020-05-17')
# routine_df.filter("app_id=20600000015411").show()
routine_df.filter("app_id=100 and country='RO' and platform=2 and device_type=1").select('app_id', 'device_type', 'country', 'platform', 'AU').show()

In [0]:

import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql import Row


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 5, 17)
    start = datetime.date(2020, 5, 17)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


date_list = get_daily_date_list()
for date in date_list:
    plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date={}/'.format(date[0])
    routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date={}/'.format(date[0])
    plproxy_df = spark.read.format("delta").load(plproxy_path)
    routine_df = spark.read.format("delta").load(routine_path)
    kpi_mapping_dict = {1: 'est_average_active_users', 2: 'est_average_session_per_user', 3: 'est_average_session_duration',
                        4: 'est_install_penetration', 5: 'est_average_active_days', 6: 'est_percentage_active_days',
                        7: 'est_average_bytes_per_user', 8: 'est_average_time_per_user', 9: 'est_usage_penetration',
                        10: 'est_open_rate', 17: 'est_average_bytes_per_session', 23: 'est_installs', 24: 'est_share_of_users',
                        25: 'est_share_of_installs'}
    for i,kpi in kpi_mapping_dict.items():
        # plproxy_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
        # routine_count =  routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
        plproxy_df = plproxy_df.withColumn(kpi, functions.UserDefinedFunction(lambda x: format(x, '.6f'))(plproxy_df[kpi]))
        routine_df = routine_df.withColumn(kpi, functions.UserDefinedFunction(lambda x: format(x, '.6f'))(routine_df[kpi]))
        plproxy_df.createOrReplaceTempView("plproxy_df")
        routine_df.createOrReplaceTempView("routine_df")
        subtract_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi).subtract(routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi)).count()
        print date[0], kpi, subtract_count
        if subtract_count!=0:
            print float(subtract_count) / plproxy_count
        sql = """
                select
                    plproxy_df.app_id, plproxy_df.device_code, plproxy_df.country_code,
                    plproxy_df.{kpi} as v3_{kpi},
                    v5.{kpi} as v5_{kpi}
                from plproxy_df inner join
                    (select app_id, device_code, country_code, {kpi}
                     from routine_df
                     ) AS v5
                on (plproxy_df.app_id=v5.app_id) and (plproxy_df.device_code=v5.device_code) and (plproxy_df.country_code=v5.country_code) and (plproxy_df.{kpi}!=v5.{kpi})
                """.format(kpi=kpi)
        unified_v1 = spark.sql(sql)
        unified_v1 = unified_v1= unified_v1.withColumn('difference', unified_v1['v3_{}'.format(kpi)] - unified_v1['v5_{}'.format(kpi)])
        unified_v1.show(3)

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 


select count(uniqlo_id) from plproxy.execute_select_nestloop(\$proxy\$ 
    select max(app_id) as uniqlo_id
    from mu.app_daily
    where 
        date='2020-06-20'
    group by
    app_id,
    store_id,
    device_id
\$proxy\$) t (uniqlo_id BIGINT);

EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;
select date, count(1) from usage_basic_kpi_fact_v6 where granularity='daily' and date between '2020-05-24' and '2020-06-20' group by date order by date asc;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

select device_id,store_id,date,app_id,kpi,estimate from plproxy.execute_select_nestloop(\$proxy\$
    select device_id,store_id,date,app_id,kpi,estimate
    from mu.app_daily 
    where
        date = '2020-06-14'
\$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) limit 50;

EOF


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;
select * from usage_basic_kpi_fact_v6 where granularity='daily' and date='2020-06-14' and app_id=284220417 and device_code='ios-phone' and country_code='AU';
EOF

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/

In [0]:

import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql import Row


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 5, 17)
    start = datetime.date(2020, 5, 17)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


date_list = get_daily_date_list()
for date in date_list:
    print date[0]
    plproxy_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date={}/'.format(date[0])
    routine_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date={}/'.format(date[0])
    plproxy_df = spark.read.format("delta").load(plproxy_path)
    routine_df = spark.read.format("delta").load(routine_path)
    kpi_mapping_dict = {1: 'est_average_active_users', 2: 'est_average_session_per_user', 3: 'est_average_session_duration',
                        4: 'est_install_penetration', 5: 'est_average_active_days', 6: 'est_percentage_active_days',
                        7: 'est_average_bytes_per_user', 8: 'est_average_time_per_user', 9: 'est_usage_penetration',
                        10: 'est_open_rate', 17: 'est_average_bytes_per_session', 23: 'est_installs', 24: 'est_share_of_users',
                        25: 'est_share_of_installs'}
    for i,kpi in kpi_mapping_dict.items():
        # plproxy_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
        # routine_count =  routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).count()
        plproxy_df = plproxy_df.withColumn(kpi, functions.UserDefinedFunction(lambda x: format(x, '.0f'))(plproxy_df[kpi]))
        routine_df = routine_df.withColumn(kpi, functions.UserDefinedFunction(lambda x: format(x, '.0f'))(routine_df[kpi]))
        plproxy_df.createOrReplaceTempView("plproxy_df")
        routine_df.createOrReplaceTempView("routine_df")
        subtract_count = plproxy_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi).subtract(routine_df.filter("est_average_active_users!=0 and {}!=0".format(kpi)).select('app_id', 'country_code', 'device_code', kpi)).count()
        print kpi, '\t', float(subtract_count) / plproxy_count
        if subtract_count!=0:
            print float(subtract_count) / plproxy_count
            sql = """
                select
                    plproxy_df.app_id, plproxy_df.device_code, plproxy_df.country_code,
                    plproxy_df.{kpi} as v3_{kpi},
                    v5.{kpi} as v5_{kpi}
                from plproxy_df inner join
                    (select app_id, device_code, country_code, {kpi}
                     from routine_df
                     ) AS v5
                on (plproxy_df.app_id=v5.app_id) and (plproxy_df.device_code=v5.device_code) and (plproxy_df.country_code=v5.country_code) and (plproxy_df.{kpi}!=v5.{kpi})
                """.format(kpi=kpi)
            unified_v1 = spark.sql(sql)
            unified_v1 = unified_v1= unified_v1.withColumn('difference', unified_v1['v3_{}'.format(kpi)] - unified_v1['v5_{}'.format(kpi)])
            unified_v1.show(3)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$
    select date, count(*) as count_a
    from mu.app_{}
    where
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 12, 31)
    start = datetime.date(2018, 4, 1)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/" \
                    "granularity={raw_granularity}/month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy(
            'date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(
            _granularity, start_day=month_day_list_tuple[1][0],
            end_day=month_day_list_tuple[1][len(month_day_list_tuple[1]) - 1]))
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            test_result.append((_granularity, dump_count['date'], plproxy_count[1], dump_count['count']))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'plproxy_count', 'dump_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0616/daily/",
            mode="append",
            partitionBy=["type"])
    retry(write_test_result, (df_write_result,), {}, interval=10)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

from pyspark.sql import Row
import random
from pyspark.sql.functions import lit, coalesce
from pyspark.sql.types import DoubleType
import datetime
from dateutil.relativedelta import relativedelta

test_result = []
device_code_agg_mapping = {'android-phone': 'android-all', 'android-tablet': 'android-all',
                           'ios-phone': 'ios-all', 'ios-tablet': 'ios-all'}


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 4, 30)
    start = datetime.date(2020, 1, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 5, 23)
    start = datetime.date(2020, 1, 4)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 12, 31)
    start = datetime.date(2018, 4, 1)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_not_empty(df, date):
    empty_count = df.filter("est_average_active_users is null").count()
    if empty_count != 0:
        print "AU is Not Empty Test Fail!!! empty_count: {}, date: {}".format(empty_count, date)
    else:
        print "AU is Not Empty Test Pass! date: {}".format(date)


def check_percentage_accuracy(df, date):
    illegal_percentage_count = df.filter(
        "est_install_penetration>1 or est_open_rate>1 or est_percent_of_wifi_total>1 "
        "or est_percentage_active_days>1 or est_share_of_category_bytes>1 or est_share_of_category_session>1 "
        "or est_share_of_category_time>1 or est_share_of_installs>1 or est_share_of_users>1 "
        "or est_usage_penetration>1").count()
    if illegal_percentage_count != 0:
        print "Percentage<1 Test Fail!!! illegal_percentage_count: {}, date: {}".format(illegal_percentage_count, date)
    else:
        print "Percentage<1 Test Pass! date: {}".format(date)


def check_usage_unified_v1_v3_accuracy(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        unified_v1_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/' \
                          'granularity={v1_granularity}/date={v1_date}/'
        unified_v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                          'granularity={v3_granularity}/date={v3_date}/'
        sample_index = random.randint(0, len(month[1]) - 1)
        date = month[1][sample_index]
        unified_v1_path_parse = unified_v1_path.format(v1_granularity=_granularity, v1_date=date)
        unified_v3_path_parse = unified_v3_path.format(v3_granularity=_granularity, v3_date=date)

        unified_v1 = spark.read.parquet(unified_v1_path_parse)
        unified_v1 = (
            unified_v1
            .drop("est_mb_per_second", "est_share_of_device_time", "est_share_of_device_session",
                  "est_share_of_device_mb", "est_panel_size")
            .withColumn('est_total_time',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_time_per_user'] / 60)
            .withColumn('est_average_time_per_user', coalesce(unified_v1['est_average_time_per_user'],
                                                              unified_v1['est_average_session_duration'] *
                                                              unified_v1['est_average_session_per_user']) * 1000)
            .withColumn('est_average_session_duration', unified_v1['est_average_session_duration'] * 1000)
        )
        unified_v1 = (
            unified_v1
            .withColumn('est_average_bytes_per_session', unified_v1['est_average_bytes_per_session'] * 1024 * 1024)
            .withColumn('est_average_bytes_per_user', unified_v1['est_average_bytes_per_user'] * 1024 * 1024)
            .withColumn('est_share_of_category_bytes', unified_v1['est_share_of_category_bytes'] * 1024 * 1024)
            .withColumn('est_install_base',
                        unified_v1['est_install_penetration'] *
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumn('est_population',
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumnRenamed('est_average_active_users_country_share', 'est_share_of_users')
            .withColumnRenamed('est_installs_country_share', 'est_share_of_installs')
            .withColumn('est_total_sessions',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_session_per_user'])
        )
        unified_v1.createOrReplaceTempView("v1_df")
        unified_v1 = spark.sql("""
                    select
                        v1_df.*,
                        ww.est_average_active_users as est_average_active_users_worldwide,
                        ww.est_installs as est_installs_worldwide
                    from v1_df left join
                        (select device_code, app_id, est_installs, est_average_active_users
                         from v1_df where country_code ='WW'
                         ) AS ww
                    on (v1_df.app_id=ww.app_id) and (v1_df.device_code = ww.device_code)
                    """)
        unified_v1 = unified_v1.na.fill(0).drop('_identifier')

        unified_v3 = spark.read.format("delta").load(unified_v3_path_parse).drop('_identifier', 'date', 'granularity')

        check_percentage_accuracy(unified_v3, date)
        check_not_empty(unified_v3, date)

        subtract_count = unified_v1.select(unified_v3.columns).subtract(unified_v3).count()
        subtract_count_reverse = unified_v3.select(unified_v1.columns).subtract(unified_v1).count()
        if subtract_count != 0 or subtract_count_reverse != 0:
            print 'Accuracy Test FAIL!!!! subtract count: {}, date: {}'.format(
                max(subtract_count, subtract_count_reverse), date)
        else:
            print 'Accuracy Test PASS! subtract count: {}, date: {}'.format(
                max(subtract_count, subtract_count_reverse), date)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_unified_v1_v3_accuracy(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import count
from pyspark.sql import Row
from pyspark.sql import functions
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)

sql = """select date, sum(count_a) from plproxy.execute_select_nestloop($proxy$
    select date, count(*) as count_a
    from mu.app_{}
    where
        date between '{start_day}' and '{end_day}'
    group by date
$proxy$) tpl (date DATE, count_a BIGINT) group by date order by date asc;"""


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 05, 24)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_usage_plproxy_dump_completeness(_granularity, date_list):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month_day_list_tuple in date_list:
        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/" \
                    "granularity={raw_granularity}/month={raw_month}/"
        dump_path_parse = dump_path.format(raw_month=month_day_list_tuple[0], raw_granularity=_granularity)
        dump_count_list = spark.read.parquet(dump_path_parse).groupBy(
            'date').agg(count('app_id').alias('count')).collect()
        dump_count_list = sorted(dump_count_list, key=lambda x: x['date'], reverse=False)

        plproxy_count_list = query(aa_dsn, sql.format(
            _granularity, start_day=month_day_list_tuple[1][0],
            end_day=month_day_list_tuple[1][len(month_day_list_tuple[1]) - 1]))
        for dump_count, plproxy_count in zip(dump_count_list, plproxy_count_list):
            if dump_count['count'] != plproxy_count[1]:
                print "Completeness Test Fail!!!! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            else:
                print "Completeness Test Pass! date: {}, plproxy_count: {}, dump_count: {}".format(
                    dump_count['date'], plproxy_count[1], dump_count['count'])
            test_result.append((_granularity, dump_count['date'], plproxy_count[1], dump_count['count']))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'plproxy_count', 'dump_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_plproxy_dump_count_0708/daily/",
            mode="append",
            partitionBy=["type"])
    retry(write_test_result, (df_write_result,), {}, interval=10)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_usage_plproxy_dump_completeness(granularity, get_path_date_list(granularity))
print 'pass'

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2020-05/")
df.filter("date='2020-05-24'").show()